In [97]:
import json
import requests as re
from datetime import datetime

import numpy as np
import pandas as pd

from dimod import ExactCQMSolver, ConstrainedQuadraticModel, QuadraticModel, Integer
from dwave.system import LeapHybridCQMSampler

In [90]:
difficulty = 'easy'
solver = 'LHCQM' #'ECQM'
time = None

with open('tokens.json') as json_file:
    token = json.load(json_file)['dwave_token']


In [91]:
with re.Session() as s:
    response = s.get('https://sugoku.herokuapp.com/board', params={'difficulty':difficulty})

    solution = solve_sudoku()

    validate = s.post('https://sugoku.herokuapp.com/validate', response.json())


TypeError: solve_sudoku() missing 3 required positional arguments: 'board', 'solver', and 'token'

In [92]:
validate.json()

{'status': 'unsolved'}

In [ ]:
now = str(datetime.now()).replace(' ','_')

In [337]:
def solve_sudoku(board, solver, token, time):
    s = len(board)
    b = int(np.sqrt(s))
    boxes = [(x, y) for x in range(b) for y in range(b)]

    cqm = ConstrainedQuadraticModel()

    fields = [[Integer(f'field_{i}_{j}', lower_bound=1, upper_bound=s) for j in range(s)] for i in range(s)]
    
    for i in range(s): # preset values must be correct
        for j in range(s):
            if board[i][j] != 0:
                cqm.add_constraint(fields[i][j] == board[i][j])
    '''
    for row in np.array(fields): # row sum must be correct
        cqm.add_constraint(sum(row) == sum(range(s+1)))

    for col in np.array(fields)[:]: # column sum must be correct
        cqm.add_constraint(sum(col) == sum(range(s+1)))
    
    for i in range(b):  # boxes must be correct
        for j in range(b):
            cqm.add_constraint(sum([fields[i*b+box[0]][j*b+box[1]] for box in boxes]) == sum(range(s+1)))
    '''
    for i in range(s):
        for j in range(s):
            for x in range(s):
                if x != j:
                    cqm.add_constraint((fields[i][j]-fields[i][x])**2 >= 1)
                
                if x != i:
                    cqm.add_constraint((fields[i][j]-fields[x][j])**2 >= 1)
                

    
    cqm.set_objective(sum(list(np.array(fields).flatten())))
    cqm.substitute_self_loops()
    
    sampler = LeapHybridCQMSampler(token=token)

    sampleset = sampler.sample_cqm(cqm).filter(lambda row: row.is_feasible)

    best = sampleset.filter(lambda row: row.is_feasible == True).first

    result = np.zeros((s, s), dtype=int)
    for i in range(s):
        for j in range(s):
            result[i][j] = best.sample[f'field_{i}_{j}']
    
    return result.tolist()
    

In [338]:
test = solve_sudoku(response.json()['board'], solver, token, time)

ValueError: SampleSet is empty

In [ ]:
test

[[4, 9, 6, 2, 8, 5, 1, 7, 3],
 [1, 2, 8, 4, 7, 5, 6, 3, 9],
 [5, 8, 4, 3, 7, 2, 1, 9, 6],
 [8, 6, 4, 3, 5, 1, 7, 9, 2],
 [7, 5, 4, 2, 9, 8, 1, 6, 3],
 [2, 8, 1, 7, 9, 4, 3, 6, 5],
 [7, 3, 1, 6, 8, 2, 9, 4, 5],
 [8, 3, 1, 5, 9, 4, 7, 2, 6],
 [9, 3, 2, 5, 4, 6, 1, 8, 7]]

In [195]:
response.json()['board']

[[4, 9, 6, 0, 0, 0, 0, 0, 3],
 [1, 2, 0, 4, 0, 0, 0, 0, 9],
 [5, 0, 0, 0, 0, 0, 1, 0, 6],
 [0, 0, 4, 3, 0, 0, 7, 9, 0],
 [0, 5, 0, 0, 9, 8, 0, 6, 0],
 [0, 8, 0, 7, 0, 4, 3, 0, 5],
 [7, 3, 1, 6, 0, 2, 9, 0, 0],
 [8, 0, 0, 0, 0, 0, 0, 2, 0],
 [9, 0, 2, 5, 4, 0, 0, 0, 7]]